In [1]:
import pandas as pd

import numpy as np

from IPython.display import display

pd.__version__ , np.__version__ #,  matplotlib.__version__, sns.__version__

('0.23.3', '1.14.2')

In [2]:
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, KFold, StratifiedKFold, \
                                    ShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_predict

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.externals import joblib

In [3]:
pwd

'/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/level_3_ensembling'

In [4]:
cd '/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/'

/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk


In [5]:
from Home_Credit_package.master_pipeline import master_pipeline

In [6]:
save_path = 'saved_models/level_3_models/'

# load this

## load df

In [7]:
o_path = 'wrangling/TRAINING_DATA_create_final_wrangled_csv/'
L1_mf_path = 'level_2_ensembling/'
original_cleaned_df = pd.read_csv(o_path + 'complete_initial_wrangled_data.csv', index_col = 'SK_ID_CURR')
level_2_metafeatures_df = pd.read_csv(L1_mf_path + 'FINAL_level_2_meta_features_df.csv', index_col = 'SK_ID_CURR')
total_df = pd.merge(original_cleaned_df, level_2_metafeatures_df, left_index=True, right_index=True, how = 'outer' )

In [8]:
total_df.info(verbose = True, null_counts = True);

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 92 columns):
TARGET                                307511 non-null int64
NAME_CONTRACT_TYPE                    307511 non-null object
CODE_GENDER                           307511 non-null object
FLAG_OWN_CAR                          307511 non-null object
FLAG_OWN_REALTY                       307511 non-null object
CNT_CHILDREN                          307511 non-null int64
NAME_EDUCATION_TYPE                   307511 non-null object
NAME_FAMILY_STATUS                    307511 non-null object
DAYS_BIRTH                            307511 non-null int64
FLAG_EMP_PHONE                        307511 non-null int64
FLAG_WORK_PHONE                       307511 non-null int64
FLAG_PHONE                            307511 non-null int64
FLAG_EMAIL                            307511 non-null int64
CNT_FAM_MEMBERS                       307511 non-null int64
REGION_RATING_CLIENT                  

In [9]:
# CHECK: this should be empty if everything is non null
total_df.isnull().any()[total_df.isnull().any()==True]

Series([], dtype: bool)

# models with only EXT SOURCES + level 2 final scores.

In [10]:
minimal_feats = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3',
                 'pwr_rescale_RanFor_EXTpoly_Level2',     
                    'pwr_rescale_LogReg_EXTpoly_Level2',     
                    'pwr_rescale_MLP_EXTpoly_Level2',        
                    'pwr_rescale_RanFor_AllFeats_Level2', 
                    'pwr_rescale_LogReg_AllFeats_Level2',  
                    'pwr_rescale_MLP_AllFeats_Level2',
                    ]

In [11]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df[minimal_feats],
                                                                                 int_cutoff=20, 
                                                                                 poly_deg=4, 
                                                                                 feats_with_interaction=[]
                                                                                 )

Total original feat len is 9. Sum of feats is 9.
Final array is length 45. final feature list is length 45.


### AdaBoost + log reg.

In [12]:
my_LgRg_4_Ada = LogisticRegression(penalty= 'l2', 
                                   random_state = 0, 
                                   class_weight = None,
                                   C= 1000,
                                   ) 

my_AdaBC_LogReg_minimal_feats = AdaBoostClassifier(base_estimator = my_LgRg_4_Ada, 
                                                   n_estimators = 50, 
                                                   algorithm = 'SAMME.R', 
                                                   random_state = 0,
                                                  learning_rate=1,
                                                  )

my_AdaBC_LogReg_minimal_feats.fit(X = total_df_piped,
                                 y = total_df['TARGET'], 
                                 )


AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=1, n_estimators=50, random_state=0)

In [13]:
final_training_predictions = my_AdaBC_LogReg_minimal_feats.predict_proba(X = total_df_piped)

In [14]:
val_scores = [x[1] for x in final_training_predictions]
roc_auc_score(total_df['TARGET'], val_scores)

0.7744626146708214

In [15]:
# fit and save model for predictions
my_AdaBC_LogReg_minimal_feats.fit(X = total_df_piped,
                                  y = total_df['TARGET'])

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=1, n_estimators=50, random_state=0)

In [16]:
joblib.dump(my_AdaBC_LogReg_minimal_feats, save_path + 'my_AdaBC_LogReg_minimal_feats_level_3.joblib')

['saved_models/level_3_models/my_AdaBC_LogReg_minimal_feats_level_3.joblib']

# models with all features

In [17]:
feats_to_drop = ['pwr_rescale_RanFor_EXTpoly_Level2',     
                 'pwr_rescale_LogReg_EXTpoly_Level2',     
                 'pwr_rescale_MLP_EXTpoly_Level2',        
                ]

In [18]:
total_df_piped, final_feature_list, total_pipeline, trans_list = \
master_pipeline(df_in = total_df.drop(columns = feats_to_drop),
               int_cutoff=20, 
               poly_deg=4, 
               feats_with_interaction=[]
               )

Total original feat len is 88. Sum of feats is 88.
Final array is length 433. final feature list is length 433.


### Adaboost + log reg.

In [19]:
my_LgRg_4_Ada = LogisticRegression(penalty= 'l2', 
                                   random_state = 0, 
                                   class_weight = None,
                                   C= 200,
                                  ) 

my_AdaBC_LogReg_all_feats = AdaBoostClassifier(base_estimator = my_LgRg_4_Ada, 
                                               n_estimators = 50, 
                                               algorithm = 'SAMME.R', 
                                               random_state = 0,
                                               learning_rate = .01,)


my_AdaBC_LogReg_all_feats.fit(X = total_df_piped,
                              y = total_df['TARGET'])


AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=200, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=0.01, n_estimators=50, random_state=0)

In [20]:
final_training_predictions = my_AdaBC_LogReg_all_feats.predict_proba(X = total_df_piped)

In [21]:
train_scores = [x[1] for x in final_training_predictions]
roc_auc_score(total_df['TARGET'], train_scores)

0.7739918994168435

In [22]:
# fit and save model for predictions
my_AdaBC_LogReg_all_feats.fit(X = total_df_piped,
                              y = total_df['TARGET'])

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=200, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=0.01, n_estimators=50, random_state=0)

In [23]:
joblib.dump(my_AdaBC_LogReg_all_feats, save_path + 'my_AdaBC_LogReg_all_feats_level_3.joblib')

['saved_models/level_3_models/my_AdaBC_LogReg_all_feats_level_3.joblib']